In [1]:
import copy
import torch
import numpy as np
import warnings

from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from data.dataloader import dataloader_info
from utils.utils import load_yaml_config, instantiate_from_config
from models.predictor import GRU
from data.dataloader import dataloader_info

from utils.experiments import train_model, evaluate_model_nonstationary, print_score, visualize_prediction

warnings.filterwarnings("ignore")
device = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
# Load configurations
configs = load_yaml_config("configs/exp1_stock_non_st.yaml")

# Initialize Diffusion_TS Model
diffusion_ts = instantiate_from_config(configs['model']).to(device)
load_model_path = "check_points/exp1_stock_non_st/DiffusionTS_10000.pth"
diffusion_ts.load_state_dict(torch.load(load_model_path))

n_sample=3000

In [3]:
dl_info_train = dataloader_info(configs, train=True)
dl_train = dl_info_train["dataloader"]
ds_train = dl_info_train["dataset"]

In [4]:
dl_info_test = dataloader_info(configs, train=False)
dl_test = dl_info_test["dataloader"]
ds_test = dl_info_test["dataset"]

In [ ]:
# train a baseline predictor
predictor_base = GRU(input_dim=5, hidden_dim=64, output_dim=1, num_layers=2, dropout=0).to(device)
optimizer_base = Adam(predictor_base.parameters(), lr=1e-3)
lossfn = nn.MSELoss()

train_model(predictor_base, 
            dl_train, 
            lossfn, 
            optimizer_base, 
            num_epochs=2000, 
            description="Baseline",
            device=device)
mae, rmse, pred_y_train, true_y_train = evaluate_model_nonstationary(predictor_base, dl_train, device, "Train")
base_mae, base_rmse, pred_y, true_y = evaluate_model_nonstationary(predictor_base, dl_test, device, "Test")


Baseline loss: 0.000100: 100%|██████████| 2000/2000 [01:04<00:00, 31.09it/s]


Train : MAE loss: 0.0744 	 RMSE Loss : 0.2705
Test : MAE loss: 1.8645 	 RMSE Loss : 3.7374


In [ ]:
# only synthetic
syn_score = []
for e in range(5):
    predictor_base_copy = copy.deepcopy(predictor_base)
    optimizer_base_copy = Adam(predictor_base_copy.parameters(), lr=1e-3)

    # additional training on synthetic data
    synthetic_data = diffusion_ts.generate_mts(batch_size=n_sample)
    synthetic_data = TensorDataset(torch.from_numpy(synthetic_data))
    dl_synthetic = DataLoader(synthetic_data, batch_size=dl_train.batch_size, shuffle=True)
    train_model(predictor_base_copy, 
                dl_synthetic, 
                lossfn, 
                optimizer_base_copy, 
                num_epochs=2000, 
                description="Synthetic",
                device=device)
    
    mae, rmse, pred_y, true_y = evaluate_model_nonstationary(predictor_base_copy, dl_test, device, "Synthetic")
    syn_score.append([mae, rmse])


Synthetic loss: 0.000028: 100%|██████████| 2000/2000 [00:46<00:00, 43.01it/s]


Synthetic : MAE loss: 1.8308 	 RMSE Loss : 3.1257


Synthetic loss: 0.000210: 100%|██████████| 2000/2000 [00:46<00:00, 43.40it/s]


Synthetic : MAE loss: 1.8119 	 RMSE Loss : 3.1073


Synthetic loss: 0.000039: 100%|██████████| 2000/2000 [00:46<00:00, 43.24it/s]


Synthetic : MAE loss: 1.8428 	 RMSE Loss : 3.3552


Synthetic loss: 0.000332: 100%|██████████| 2000/2000 [00:46<00:00, 43.36it/s]


Synthetic : MAE loss: 1.8500 	 RMSE Loss : 3.6025


Synthetic loss: 0.000010: 100%|██████████| 2000/2000 [00:47<00:00, 42.53it/s]


Synthetic : MAE loss: 1.7438 	 RMSE Loss : 2.9208


In [ ]:
*_, = print_score(syn_score, base_mae, base_rmse)


        MAE       MSE
0  1.830766  3.125695
1  1.811938  3.107343
2  1.842784  3.355216
3  1.849998  3.602454
4  1.743834  2.920814
----------------------------------------------------
baseline mae : 1.8645, baseline rmse : 3.7374
MAE : 1.8159(0.0383)(-2.6096%) 
MSE : 3.2223(0.2348)(-13.7829%)
----------------------------------------------------


(1.8158642292022704,
 0.03825070082376774,
 -2.6095888606748123,
 3.222304582595825,
 0.23478725844973808,
 -13.782863711623852)

In [ ]:
# only origin
origin_score = []
for e in range(5):
    predictor_base_copy = copy.deepcopy(predictor_base)
    optimizer_base_copy = Adam(predictor_base_copy.parameters(), lr=1e-3)

    # additional training on original data
    train_model(predictor_base_copy, 
                dl_train, 
                lossfn, 
                optimizer_base_copy, 
                num_epochs=2000, 
                description="Origin",
                device=device)
    mae, rmse, pred_y, true_y = evaluate_model_nonstationary(predictor_base_copy, dl_test, device, "Origin")
    origin_score.append([mae, rmse])

Origin loss: 0.000069: 100%|██████████| 2000/2000 [01:07<00:00, 29.70it/s]


Origin : MAE loss: 1.6620 	 RMSE Loss : 2.7489


Origin loss: 0.000177:  16%|█▋        | 326/2000 [00:11<00:57, 29.28it/s]

In [ ]:
*_, = print_score(origin_score, base_mae, base_rmse)

In [ ]:
# origin+synthetic
ori_syn_score = []
for e in range(5):
    predictor_base_copy = copy.deepcopy(predictor_base)
    optimizer_base_copy = Adam(predictor_base_copy.parameters(), lr=1e-3)

    # additional training on ori+syn data
    idx = np.random.permutation(len(ds_train))[:int(n_sample/2)]
    origin_data = ds_train.data_norm[idx]
    synthetic_data = diffusion_ts.generate_mts(batch_size=int(n_sample/2))
    ori_syn_data = np.concatenate([origin_data, synthetic_data])
    ori_syn_data = TensorDataset(torch.from_numpy(ori_syn_data))
    dl_ori_syn = DataLoader(ori_syn_data, batch_size=dl_train.batch_size, shuffle=True)
    train_model(predictor_base_copy, 
                dl_ori_syn, 
                lossfn, 
                optimizer_base_copy, 
                num_epochs=2000, 
                description="Ori+Syn",
                device=device)
    mae, rmse, pred_y, true_y = evaluate_model_nonstationary(predictor_base_copy, dl_test, device, "Ori+Syn")
    ori_syn_score.append([mae, rmse])


In [ ]:
*_, = print_score(ori_syn_score, base_mae, base_rmse)